In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import json
from itertools import chain
from rich import print


In [3]:
with open("../../dataset\\info_hotel\\feature_sub_rate.json", 'r', encoding='utf-8') as f:
    feature_sub_rate = json.load(f)

with open("../../dataset\\sub_model\\weights_criteria_utilities.json", 'r', encoding='utf-8') as f:
    weights_criteria_utilities = json.load(f)

with open("../../dataset\\info_hotel\\feature_popular_facilities.json", 'r', encoding='utf-8') as f:
    feature_popular_facilities = json.load(f)
    
with open("../../dataset\\info_hotel\\feature_facilities.json", 'r', encoding='utf-8') as f:
    feature_facilities = json.load(f)

with open('../../dataset\\info_hotel\\feature_url.json', 'r', encoding='utf-8') as f:
    feature_url = json.load(f)

with open("../../dataset\\info_hotel\\feature_allRoom.json", 'r', encoding='utf-8') as f:
    feature_allRoom = json.load(f)

with open("../../dataset\\info_hotel\\feature_location.json", 'r', encoding='utf-8') as f:
    feature_location = json.load(f)

with open("../../dataset\\info_hotel\\feature_policies.json", 'r', encoding='utf-8') as f:
    feature_policies = json.load(f)


In [3]:
print(len(feature_sub_rate))
print(len(feature_popular_facilities))
print(len(feature_facilities))
print(len(feature_url))
print(len(feature_allRoom))


11486

11486

11486

11486

42983

In [3]:
def convert_amenity(value):
    amenities = {
        1: "Tiện ích phòng",
        2: "Tiện ích công nghệ & kết nối",
        3: "Tiện ích phòng tắm",
        4: "Dịch vụ ẩm thực",
        5: "Tiện ích thư giãn & giải trí",
        6: "Tiện ích dành cho trẻ em",
        7: "Dịch vụ khách sạn",
        8: "Dịch vụ bảo mật & an toàn",
        9: "Dịch vụ tổ chức sự kiện",
        10: "Tiện nghi hỗ trợ đặc biệt",
        11: "Dịch vụ giải trí ngoài trời",
        12: "Dịch vụ bãi đỗ xe",
        13: "Tiện ích thiên nhiên",
        14: "Hệ thống điều hòa & sưởi ấm",
        15: "Cơ sở vật chất chung",
        16: "Dịch vụ chăm sóc sức khỏe & spa",
        17: "Dịch vụ đưa đón & phương tiện di chuyển",
        18: "Tiện ích công việc"
    }
    
    if isinstance(value, int):
        return amenities.get(value, "Không tìm thấy")
    elif isinstance(value, str):
        return next((k for k, v in amenities.items() if v == value), "Không tìm thấy")
    return "Định dạng không hợp lệ"

In [4]:
import sys
import os

# Đường dẫn đến thư mục mini_model
mini_model_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Lấy danh sách tất cả thư mục con trong mini_model
for folder in os.listdir(mini_model_path):
    folder_path = os.path.join(mini_model_path, folder)
    # Chỉ thêm nếu là thư mục và không phải file
    if os.path.isdir(folder_path):
        if folder_path not in sys.path:
            sys.path.append(folder_path)

# Import module
# import get_score_sub_rate as gssr
# import get_score_amenities as gsa
# import json

In [5]:
import location_recommend 
import policies_recommend

In [7]:
best_hotels[0]

{'province/ city': 'Ninh Bình',
 'nearby_places': [{'title': 'Xung quanh có gì?',
   'detail': {'Cuc Phuong National Park': '0 m'}},
  {'title': 'Nhà hàng & quán cà phê',
   'detail': {'Cafe/quán bar ❂ Karaoke Đôn Phương': '12 km',
    'Nhà hàng ❂ Nhà Hàng Quý Hiệp': '13 km',
    'Nhà hàng ❂ Nhà Hàng Hoàng Giang - Ninh Bình': '21 km'}},
  {'title': 'Các sân bay gần nhất', 'detail': {'Sân bay Thọ Xuân': '61 km'}},
  {'title': 'Các trung tâm gần nhất',
   'detail': {'Thành phố Ninh Bình, Ninh Bình': 18.27,
    'Thành phố Tam Điệp, Ninh Bình': 10.4}},
  {'title': 'Các bến xe khách & bến cảng gần nhất',
   'detail': {'Bến xe khách Ninh Bình': 18.31,
    'Bến xe Nho Quan': 14.53,
    'Bến xe Kim Sơn': 29.62,
    'Bến xe Tam Điệp': 4.83}},
  {'title': 'Các trung tâm gần nhất',
   'detail': {'Thành phố Ninh Bình, Ninh Bình': 18.27,
    'Thành phố Tam Điệp, Ninh Bình': 10.4}},
  {'title': 'Các bến xe khách & bến cảng gần nhất',
   'detail': {'Bến xe khách Ninh Bình': 18.31,
    'Bến xe Nho Qua

In [8]:
# Input ví dụ
user_places = []
province = "Ninh Bình"
is_near_center = True


best_hotels = location_recommend.find_hotels_near_location(feature_location, user_places, location = province, max_distance_km=20)
# Thêm điểm và xếp hạng
ranked_hotels =  location_recommend.get_location_score(best_hotels, user_places, province, is_near_center)

# In kết quả
for hotel in ranked_hotels[:5]:  # Top 5 khách sạn
    print('Score:', hotel['location_score'])

Score: 0.265

Score: 0.265

Score: 0.265

Score: 0.265

Score: 0.165

In [6]:
user_input = {
    # Input địa điểm
    'province': 'TP Hồ Chí Minh',
    'district': None,
    'nearby_places': ["Chợ Bến Thành", "Dinh Độc Lập", "Nhà thờ Đức Bà"], # Bao gồm địa điểm tham quan + địa điểm di chuyển
    'is_near_center': True,
    # Input thông tin tổng quát
    'stars_rating': 3,
    'price': (800000, 2000000),
    'rating': 8,
    'sub_rate': {
        "staff": 9.0,
        "facilities": 8.3,
        "cleanliness": 8.7,
        "comfort": 8.7,
        "value": 8.4,
        "location": 8.4,
        "free WiFi": 8.3
    },
    # Input thông tin về cơ sở vật chất và tiện nghi
    'services': [],  # 1 trong 18 loại dịch vụ được giới thiệu trước
    'amenities': ["TV màn hình phẳng, két an toàn"],
    # Input thông tin về phòng
    'capacity': 2,
    'room_type': 'Family',
    'area': 50,
    'bed_type': '',
    'included_breakfast': True,
    'room_amenties':[],
    'room_facilities':[],
    'room_view': [],

    # Input thông tin về chính sách
    'policies': {
        "Nhận phòng": '11:00-12:00',
        "Trả phòng": '12:00-13:00',
        "Chỉ thanh toán bằng tiền mặt": None,
        "Các phương thức thanh toán được chấp nhận": 'Bankcard',
        "Giới hạn độ tuổi": None,
        "Giờ giới nghiêm": 'Cổng vào chỗ nghỉ sẽ đóng trong khoảng 00:00-5:00',
        "Hút thuốc": 'Không cho phép hút thuốc.',
        "Hủy đặt phòng/ Trả trước": 'Các chính sách hủy và thanh toán trước',
        "Không giới hạn độ tuổi": 'Không có yêu cầu về độ tuổi',
        "Nhóm": None,
        "Thẻ được chấp nhận tại chỗ nghỉ này": None,
        "Thẻ được chấp nhận tại khách sạn này": 'Bankcard',
        "Thời gian yên lặng": None,
        "Tiệc tùng": 'Cho phép tổ chức',
        "Trẻ em và giường": 'Trẻ em dưới 5 tuổi',
        "Vật nuôi": 'Cho phép mang vật nuôi',
        "Đặt cọc đề phòng hư hại có thể hoàn lại": 'Yêu cầu VND 1.000.000 tiền đặt cọc đề phòng hư hại khi đến nghỉ.',
        'room_service_included': 'Không cần thanh toán trước - thanh toán tại chỗ nghỉ\nKhông cần thẻ tín dụng'
    }

    # Input thông tin quan tâm về review

}

In [7]:
# 1. Filter địa điểm
if user_input.get('nearby_places') or user_input.get('province'):
    filter_location_hotels = location_recommend.find_hotels_near_location(
        feature_location, user_input.get('nearby_places', []), user_input.get('province'), max_distance_km=20
    )
    result_location_hotels_score = location_recommend.get_location_score(
        filter_location_hotels, user_input.get('nearby_places', []),
        user_input.get('province'), user_input.get('is_near_center', False)
    )
else:
    result_location_hotels_score = feature_location
    for hotel in result_location_hotels_score:
        hotel['location_score'] = 0

In [8]:
len(result_location_hotels_score)

890

In [ ]:
policies_recommend

In [9]:
ids_result = []
for id in result_location_hotels_score:
    ids_result.append(id['id'])

In [10]:
ids_result

['257543',
 '264198',
 '10128837',
 '294771',
 '2597342',
 '11971962',
 '264986',
 '1312425',
 '10012516',
 '10052199',
 '10059536',
 '10059845',
 '10112773',
 '10121408',
 '10175818',
 '10230604',
 '10257028',
 '10257358',
 '10322238',
 '10335019',
 '10367451',
 '10367890',
 '10391254',
 '10410565',
 '10424009',
 '10474624',
 '10505705',
 '10527088',
 '10533390',
 '10577278',
 '10581778',
 '10600192',
 '10618677',
 '10636046',
 '1063721',
 '10668578',
 '10734309',
 '10868832',
 '1087399',
 '10874751',
 '10915959',
 '10941335',
 '10967940',
 '10973516',
 '10978493',
 '1097973',
 '10980611',
 '1102388',
 '11058407',
 '11061532',
 '1108901',
 '11099373',
 '11108037',
 '11110404',
 '11161251',
 '11186552',
 '11192005',
 '11192379',
 '11203515',
 '11205316',
 '11205740',
 '11212692',
 '11219033',
 '11223889',
 '11239396',
 '11242837',
 '11257100',
 '11259332',
 '11265827',
 '11278843',
 '11279232',
 '11280666',
 '11284666',
 '11286914',
 '11303700',
 '11320708',
 '11334623',
 '11336786',
 

In [11]:
filtered_policies = {}
for id in ids_result:
    if id in feature_policies:
        filtered_policies[id] = feature_policies[id]
    else:
        print(f"ID {id} không có trong feature_policies")

In [12]:
result = {}# 4. Chính sách
if user_input.get('policies'):
    for id, data in filtered_policies.items():
        result[id] = policies_recommend.find_similar_hotel_policies(user_input['policies'], data)


In [13]:
result

{'257543': 0.17857142857142858,
 '264198': 0.03571428571428571,
 '10128837': 0.03571428571428571,
 '294771': 0.03571428571428571,
 '2597342': 0.03571428571428571,
 '11971962': 0.050400251966700664,
 '264986': 0.03571428571428571,
 '1312425': 0.03571428571428571,
 '10012516': 0.03571428571428571,
 '10052199': 0.03571428571428571,
 '10059536': 0.33611453768098637,
 '10059845': 0.03571428571428571,
 '10112773': 0.05040025196670068,
 '10121408': 0.014685966252414964,
 '10175818': 0.19325739482384355,
 '10230604': 0.22897168053812925,
 '10257028': 0.05040025196670068,
 '10257358': 0.014685966252414964,
 '10322238': 0.050400251966700664,
 '10335019': 0.07142857142857142,
 '10367451': 0.014685966252414953,
 '10367890': 0.014685966252414953,
 '10391254': 0.014685966252414953,
 '10410565': 0.05040025196670068,
 '10424009': 0.014685966252414953,
 '10474624': 0.014685966252414953,
 '10505705': 0.014685966252414953,
 '10527088': 0.014685966252414953,
 '10533390': 0.014685966252414953,
 '10577278':

In [14]:
def filter_matching_elements(ids_A, B):
    try:
        return [item for item in B if item['id'] in ids_A]  # Lọc các phần tử B có id trong A
    except:
        return [item for item in B if item['id_room'] in ids_A]

In [4]:
query = {
    'location': ['Thành phố Hồ Chí Minh'],
    'nearby_places': [],
    'price_range': (300000, 1000000),  
    'rating': (3, 5),
    'services': ["Dịch vụ tổ chức sự kiện"], 
    'amenities': ["TV màn hình phẳng, két an toàn"],
    'room_type': ['phòng đơn', 'phòng đôi'], # tạm chưa xử lý
    'booking_flexibility': ['hủy miễn phí', 'thanh toán khi nhận phòng'], # tạm chưa xử lý
    'distance_to_city_center': 5,
    'public_transport_access': ['ga tàu'], # đã xử lý 50%, 
    'capacity': 2, 
}


In [7]:
designated_utility = []
for amenity in query['amenities']:
    cluster = gsa.find_clusters(amenity, threshold=0.01)
    print(f"Tiện nghi: [bold #80CFFF]{amenity}[/bold #80CFFF] => Dịch vụ: [bold #80CFFF]{' và '.join(cluster)}[/bold #80CFFF]")
    designated_utility.append(cluster)

Tiện nghi: TV màn hình phẳng, két an toàn => Dịch vụ: Tiện ích thư giãn & giải trí và Tiện ích phòng

In [7]:
def processing_query(*results, weights):
    """
    Tổng hợp và chuẩn hóa điểm số từ nhiều nhóm kết quả với trọng số tương ứng.

    Parameters:
    - results: Danh sách không giới hạn số lượng kết quả [{ 'id': str, 'score': float hoặc tuple }]
    - weights: Danh sách trọng số tương ứng

    Returns:
    - Danh sách kết quả cuối cùng [{'id': str, 'final_score': float}], sắp xếp giảm dần theo điểm
    """
    if len(results) != len(weights):
        raise ValueError("Số lượng kết quả và trọng số phải bằng nhau!")

    all_ids = set()
    normalized_scores = []
    extracted_ids = []

    # Trích xuất dữ liệu & chuẩn hóa điểm số cho từng nhóm
    for result in results:
        ids, scores = gsa.extract_scores(result)
        extracted_ids.append(ids)
        normalized_scores.append(gsa.normalize_scores(scores))
        all_ids.update(ids)

    # Tính tổng điểm có trọng số
    final_scores = {id_: 0 for id_ in all_ids}

    for i, ids in enumerate(extracted_ids):
        for idx, id_ in enumerate(ids):
            final_scores[id_] += normalized_scores[i][idx] * weights[i]

    # Chuyển kết quả thành danh sách và sắp xếp theo điểm giảm dần
    final_results = [{'id': k, 'score': v} for k, v in final_scores.items()]
    final_results.sort(key=lambda x: x['score'], reverse=True)

    return final_results

In [8]:
recommend_faclities_hotel = gsa.HotelSimilarityRecommender(    
    model_name='paraphrase-multilingual-MiniLM-L12-v2',
    use_gpu=True,
    model_dir="D:\\graduate_dissertation\\final\\mini_model\\function_get_score_amenities\\model_similarity_amenities\\hotel",
    type='hotel',
    batch_size=256,
    faiss_metric='IP'
)
recommend_faclities_hotel.load_model()

recommend_faclities_room = gsa.HotelSimilarityRecommender(
    model_name='paraphrase-multilingual-MiniLM-L12-v2',
    use_gpu=True,
    model_dir="D:\\graduate_dissertation\\final\\mini_model\\function_get_score_amenities\\model_similarity_amenities\\room",
    type='room',
    batch_size=256,
    faiss_metric='IP'
)
recommend_faclities_room.load_model()



Loading Sentence-BERT model from D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\hotel\model.safetensors
Loaded model on CPU
Loaded model on CPU
Loaded hotel data from D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\hotel\hotel_data.pkl
Loaded vectors from D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\hotel\hotel_vectors.npy
Loaded Faiss index from D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\hotel\faiss_index.bin
Loading Sentence-BERT model from D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\room\model.safetensors
Loaded model on CPU
Loaded model on CPU
Loaded hotel data from D:\graduate_dissertation\final\mini_model\function_get_score_amenities\model_similarity_amenities\room\hotel_data.pkl
Loaded vectors from D:\graduate_dissertati

In [26]:
def compute_total_score(source1, source2, source3, weights=(1/3, 1/3, 1/3)):
    from collections import defaultdict

    def normalize_minmax(values_dict):
        values = list(values_dict.values())
        min_v, max_v = min(values), max(values)
        if max_v == min_v:
            return {k: 0.0 for k in values_dict}  # tránh chia 0
        return {k: (v - min_v) / (max_v - min_v) for k, v in values_dict.items()}

    # Bước 1: Chuyển các source về dict {id: score}
    def to_dict_source(source, key='score'):
        if isinstance(source, dict):
            return source
        elif isinstance(source, list):
            return {item['id']: item.get(key, 0.0) for item in source}
        else:
            raise ValueError("Unsupported data format")

    s1_dict = to_dict_source(source1, key='score')
    s2_dict = to_dict_source(source2, key='final_score')
    s3_dict = to_dict_source(source3)

    # Bước 2: Chuẩn hóa từng loại score
    s1_norm = normalize_minmax(s1_dict)
    s2_norm = normalize_minmax(s2_dict)
    s3_norm = normalize_minmax(s3_dict)

    # Bước 3: Gộp điểm theo id, tính điểm tổng
    total_scores = defaultdict(float)
    all_ids = set(s1_norm) | set(s2_norm) | set(s3_norm)
    w1, w2, w3 = weights

    for id_ in all_ids:
        total = (
            s1_norm.get(id_, 0.0) * w1 +
            s2_norm.get(id_, 0.0) * w2 +
            s3_norm.get(id_, 0.0) * w3
        )
        total_scores[id_] = total

    # Sắp xếp theo điểm giảm dần
    sorted_scores = sorted(total_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_scores


In [10]:

def get_best_hotels(query): 
    """    
    from geopy.geocoders import Nominatim
    from geopy.distance import geodesic
    import numpy as np

    max_distance_km = 18

    if query['location'] and query['distance_to_city_center']:
        max_distance_km = query['distance_to_city_center']
        print( f"🔵 Thực hiện tìm kiếm khách sạn cách trung tâm thành phố {', '.join(query['location'])} {max_distance_km} km ")
        result_location = address_nearplaces_recommendation.find_hotels_near_location(places = query['location'], max_distance_km = max_distance_km + 2)
    else:
        result_location = data_list

    if query['nearby_places']:
        print( f"🔵 Thực hiện tìm kiếm khách sạn gần {', '.join(query['nearby_places'])}")
        result_location = address_nearplaces_recommendation.find_hotels_near_location(data_list = result_location, places = query['nearby_places'], max_distance_km = 20)

    ids_resulf = {item['id'] for item in result_location}  # Tạo tập hợp chứa các id từ A
    
    if query['public_transport_access']:
        print( f"🔵 Thực hiện lọc khách sạn gần {', '.join(query['public_transport_access'])}")
    result_location = filter_hotels_by_distance(filter_matching_elements(ids_resulf, feature_address_nearplaces), 10)
    ids_resulf = {item['id'] for item in result_location} 
    print( f"✅ Hoàn thành thu thập khách sạn thỏa vị trí")

    del geodesic
    del Nominatim"""

    # =====================================================================
    print( f"🔵 Xem như đã thu được danh sách các ID hotel thỏa điều kiện địa lý.")
    print( f"🔵 Xem như đã thu được danh sách các ID hotel_room thỏa điều kiện loại phòng và số lượng.")
    print( f"🔵 Xem như đã Thực hiện tính toán các khách sạn phù hợp với phân khúc giá")
    # result_price = get_score_price(filter_matching_elements(ids_resulf, feature_allRoom), 
    #                                 (float(query['price_range'][1]) - float(query['price_range'][0])) / 2)
    
    print( f"🔵 Bắt đầu tìm kiếm các khách sạn được đánh giá cao về dịch vụ của bạn.") 
    if query['services']:
        print(f"✎ Dịch vụ: [bold #80CFFF]{', '.join(query['services'])}[/bold #80CFFF].")
        result_services = gssr.get_score_sub_rate(query['services'], 
                                                  filter_matching_elements(ids_resulf, feature_sub_rate))
        # OUTPUT: [{'id': '1', 'score': 0.8}, {'id': '2', 'score': 0.7}, ...]

    else:
        print(f"✎ Dịch vụ: Rỗng")
        print(f"✎ Dựa vào các tiện ích mà bạn yêu cầu: [bold #80CFFF]{', '.join(query['amenities'])}[/bold #80CFFF].")
        designated_utility = []
        for amenity in query['amenities']:
            cluster = gsa.find_clusters(amenity, threshold=0.01)
            print(f"Tiện nghi: [bold #80CFFF]{amenity}[/bold #80CFFF] => Dịch vụ: [bold #80CFFF]{' và '.join(cluster)}[/bold #80CFFF]")
            designated_utility.append(cluster)
        designated_utility = list(set(chain(*designated_utility)))
        print(f"✎ Nên sẽ ưu tiên các khách sạn được đánh cao với các tiêu chí đã phục thuộc các tiện ích trên.")
        result_services = gssr.get_score_sub_rate(designated_utility, 
                                                  filter_matching_elements(ids_resulf, feature_sub_rate))
        # OUTPUT: [{'id': '1', 'score': 0.8}, {'id': '2', 'score': 0.7}, ...]
    
    print(" Đã tìm kiếm các khách sạn được đánh giá cao về dịch vụ của bạn.")

    if query['services']:
        print(f"🔵 Bắt đầu tìm kiếm các khách sạn có dịch vụ: [bold #80CFFF]{', '.join(query['services'])}[/bold #80CFFF]")
        score_services = gsa.get_score_services(user_input = query['services'], 
                                    List_ids = ids_resulf, 
                                    weights=[0.5, 0.5])
        
        print(f" Đã tìm kiếm các khách sạn khớp dịch vụ bạn quan tâm.")

    if query['amenities']:
        print(f"🔵 Bắt đầu tìm kiếm khách sạn chứa các tiện nghi [bold #80CFFF]{', '.join(query['amenities'])}[/bold #80CFFF]")
        result_facilities_hotel = recommend_faclities_hotel.predict_assignID(
            input_amenities= query['amenities'],
            hotel_ids=ids_resulf,
            similarity_threshold=0.9, 
            normalization_factor_base=10
        )
        result_facilities_room = recommend_faclities_room.predict_assignID(
            input_amenities= query['amenities'],
            hotel_ids=ids_resulf,
            similarity_threshold=0.9, 
            normalization_factor_base=10
        )

        facilities = gsa.calculate_hotel_scores(result_facilities_hotel, 
                                   result_facilities_room, 
                                   threshold=0.9, 
                                   weight=[0.5, 0.5])
        
        print(f" Đã tìm kiếm các khách sạn có tiện nghi bạn quan tâm.")

    print(f"Danh sách khách sạn đã tìm kiếm được:")
    result = compute_total_score(facilities, result_services, score_services, weights=(0.33, 0.33, 0.33))
    return result

In [22]:
result = get_best_hotels(query)

🔵 Xem như đã thu được danh sách các ID hotel thỏa điều kiện địa lý.

🔵 Xem như đã thu được danh sách các ID hotel_room thỏa điều kiện loại phòng và số lượng.

🔵 Xem như đã Thực hiện tính toán các khách sạn phù hợp với phân khúc giá

🔵 Bắt đầu tìm kiếm các khách sạn được đánh giá cao về dịch vụ của bạn.

✎ Dịch vụ: Dịch vụ tổ chức sự kiện.

...

Đã tìm kiếm các khách sạn khớp dịch vụ bạn quan tâm.

🔵 Bắt đầu tìm kiếm khách sạn chứa các tiện nghi TV màn hình phẳng, két an toàn

Cảnh báo: room '10000593' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10000937' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10001167' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10001434' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10001679' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10003086' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10003813' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10004054' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10006504' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10006885' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10006969' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10007210' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10007353' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '10007578' không có trong dữ liệu, trả về kết quả rỗng
Cảnh báo: room '1000

Đã tìm kiếm các khách sạn có tiện nghi bạn quan tâm.

Danh sách khách sạn đã tìm kiếm được:

In [27]:
sorted_scores = sorted(result.items(), key=lambda x: x[1], reverse=True)

In [28]:
sorted_scores

[('10034269', 0.33),
 ('1005432', 0.32783257991013676),
 ('10086551', 0.32655797017267485),
 ('10012325', 0.31211705032255627),
 ('10031751', 0.3107297347404215),
 ('10073188', 0.3072295513648391),
 ('10012226', 0.30518077251620257),
 ('10006885', 0.3044690251036159),
 ('10039464', 0.29674270600204466),
 ('10014032', 0.29623663008352386),
 ('10045249', 0.28736624662374954),
 ('10067823', 0.28728495169791746),
 ('1002711', 0.28603224957582535),
 ('10031734', 0.2846866484941894),
 ('1005271', 0.28400271074860184),
 ('10003813', 0.2793582594566764),
 ('10072786', 0.279232886029302),
 ('1001739', 0.2751487329916616),
 ('10006969', 0.2721001399280332),
 ('1003155', 0.2694922333860609),
 ('1007578', 0.265231252214012),
 ('10053483', 0.2640223653226597),
 ('10018040', 0.26392736563287467),
 ('10052842', 0.2635114543900922),
 ('10020191', 0.2610461974349572),
 ('1002738', 0.26066239735442914),
 ('10014903', 0.25977292149447373),
 ('10051917', 0.2576168053340023),
 ('10004054', 0.25735301363660